In [ ]:
import numpy as np
import pandas as pd
from Gen_data import SimulationStudy
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from econml.metalearners import SLearner, TLearner
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML, NonParamDML
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error 
import shap

Define Necessary Functions

In [ ]:
#Split into train and test set
def get_split(simulation: pd.DataFrame) -> np.ndarray:
    
    train_df, test_df = train_test_split(simulation, test_size=0.5, random_state=42)

    # Extract features and target variables for training
    X_train = train_df[[col for col in simulation.columns if col.startswith('X')]]
    T_train = train_df['T']
    Y_train = train_df['y']

    # Extract features and true CATE for testing
    X_test = test_df[[col for col in simulation.columns if col.startswith('X')]]
    T_test = test_df['T']
    y_test = test_df['y']

    true_cate_test = test_df['CATE']

    return train_df, test_df, X_train, Y_train, T_train, X_test, T_test, y_test, true_cate_test

In [ ]:
#Create OLS feature-treatment interactions
def get_ols_features(X_train: pd.DataFrame, T_train: pd.DataFrame, X_test: pd.DataFrame, T_test: pd.DataFrame) -> pd.DataFrame:  

    interactions_train = pd.DataFrame()
    interactions_test = pd.DataFrame()


    for col in X_train.columns:
        interactions_train[f'{col}_T'] = X_train[col]*T_train
        interactions_test[f'{col}_T'] = X_test[col]*T_test


    X_train_ols = X_train.join(interactions_train)
    X_test_ols = X_test.join(interactions_test)


    return X_train_ols, X_test_ols



In [ ]:
def vis_treatment_eff(simulation: pd.DataFrame):
    sorted_cate = simulation[simulation['T'] == 1]
    sorted_cate = sorted_cate.sort_values(by=['CATE'])
    sorted_cate = sorted_cate[['CATE']]

    x = np.linspace(0, len(sorted_cate), num=len(sorted_cate))
    plt.scatter(x, sorted_cate, alpha=0.5)
    #plt.plot(x, sorted_cate, color='red')
    plt.xlabel('Observations')
    plt.ylabel('Treatment Effects')

In [ ]:
def plot_predict_cate_corr(est_cate: np.array, true_cate_test: np.array):
    
    plt.figure(figsize=(8, 8))
    plt.scatter(est_cate, true_cate_test, alpha=0.5)    
    plt.plot([min(true_cate_test), max(true_cate_test)], [min(true_cate_test), max(true_cate_test)], '--', color='red')
    plt.xlabel('Estimated CATE')
    plt.ylabel('True CATE')
    plt.grid(True)
    plt.show()

    if type(est_cate) == np.ndarray:
        

    reg = LinearRegression().fit(est_cate, true_cate_test)
    print(f'Correlation between true and predicted CATE: {reg.score(est_cate, true_cate_test)}')

Create Simulations

In [ ]:
#Create simulations

sim_dict = {}
i = 1

for p in [10, 20, 30, 50]:
    for n in [5000, 7000, 10000]:
        sim: SimulationStudy = SimulationStudy(p=p, mean_correlation=0.8, cor_variance=0.01, n=n, geom=True)
        simulation = sim.create_dataset()


        key = f'Simulation_{p}_{n}'
        value = simulation

        sim_dict[key]= value
        i += 1


In [ ]:
#Split df into train and test data

splits_dict = {}

for key, value in sim_dict.items():

    train_df, test_df, X_train, Y_train, T_train, X_test, T_test, y_test, true_cate_test = get_split(value)
    
    temp_dict = {'train_df': train_df, 'test_df': test_df, 
                   'X_train': X_train, 'Y_train': Y_train, 
                   'T_train': T_train, 'X_test': X_test, 
                   'T_test': T_test, 'Y_test': y_test, 
                   'true_cate_test': true_cate_test}

        
    splits_dict[key] = temp_dict



In [ ]:
#Visualize Treatment Effects for ordered observations
vis_treatment_eff(sim_dict['Simulation_10_10000'])

OLS Model

In [ ]:
#Estimate a fully feature-treatment interacted model

x_train_ols, x_test_ols = get_ols_features(splits_dict['Simulation_10_10000']['X_train'], splits_dict['Simulation_10_10000']['T_train'],
                                            splits_dict['Simulation_10_10000']['X_test'], splits_dict['Simulation_10_10000']['T_test'])

In [ ]:
def ols_estimator(x_train_ols, y_train, x_test_ols, true_cate_test):

    #Create OLS object
    ols = LinearRegression()

    #Fit OLS 
    ols.fit(x_train_ols, y_train)

    #Estimate CATE
    x_test_ols = x_test_ols[[col for col in x_test_ols.columns if col.startswith('X') and col.endswith('T')]]

    coeff = pd.DataFrame(ols.coef_, x_train_ols.columns).T
    coeff = coeff[[col for col in coeff.columns if col.startswith('X') and col.endswith('T')]].T
    
    estimated_cate = x_test_ols.dot(coeff)
    
    #Calculate MSE
    OLS_MSE = mean_squared_error(true_cate_test, estimated_cate)

    return estimated_cate, OLS_MSE 
    

In [ ]:
estimated_cate_ols, ols_mse = ols_estimator(x_train_ols, splits_dict['Simulation_10_10000']['Y_train'], x_test_ols, splits_dict['Simulation_10_10000']['true_cate_test'])

In [ ]:
plot_predict_cate_corr(estimated_cate_ols, splits_dict['Simulation_10_10000']['true_cate_test'])

S-Learner

In [ ]:
#s_learner = SLearner(overall_model=RandomForestRegressor())  
#s_learner_model = s_learner.fit(Y=Y_train_1, T=T_train_1, X=X_train_1)
#pred_cate_s = s_learner_model.effect(X_test_1)
#plot_predict_cate_corr(pred_cate_s, true_cate_test_1)

T-Learner

In [ ]:
def TLearner_estimator(Y_train: pd.DataFrame, T_train: pd.DataFrame, X_train: pd.DataFrame, X_test: pd.DataFrame, true_cate_test: pd.DataFrame):

    Y_train = Y_train.to_numpy()
    T_train = T_train.to_numpy()
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    true_cate_test = true_cate_test.to_numpy()

    est_t = TLearner(models=RandomForestRegressor())
    est_t.fit(Y=Y_train, T=T_train, X=X_train)
    estimated_cate_t = est_t.effect(X_test)
    T_MSE = mean_squared_error(true_cate_test, estimated_cate_t)

    return estimated_cate_t, T_MSE

estimated_cate_t, T_MSE = TLearner_estimator(splits_dict['Simulation_10_10000']['Y_train'], splits_dict['Simulation_10_10000']['T_train'], 
                                             splits_dict['Simulation_10_10000']['X_train'], splits_dict['Simulation_10_10000']['X_test'],
                                             splits_dict['Simulation_10_10000']['true_cate_test'])



In [ ]:
plot_predict_cate_corr(estimated_cate_t, splits_dict['Simulation_10_10000']['true_cate_test'])

Causal Forest DML

In [ ]:
def CF_DML(Y_train: pd.DataFrame, T_train: pd.DataFrame, X_train: pd.DataFrame, X_test: pd.DataFrame, true_cate_test: pd.DataFrame) -> np.ndarray:

    Y_train = Y_train.to_numpy()
    T_train = T_train.to_numpy()
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    true_cate_test = true_cate_test.to_numpy()

#Estimate the causal forest model
    est_cfdml = CausalForestDML(model_y='auto',
                        model_t=DummyClassifier(),
                        discrete_treatment=True,
                        cv=5,
                        n_estimators=1000,
                        random_state=42,
                        criterion='mse',
                        honest=True)

    est_cfdml.fit(Y=Y_train, T=T_train, X=X_train, W=None, cache_values=True)
    estimated_cate = est_cfdml.effect(X_test)
    mse = mean_squared_error(true_cate_test, estimated_cate)
    
    return estimated_cate, mse

estimated_cate_dml, cf_dml_MSE = CF_DML(splits_dict['Simulation_10_10000']['Y_train'], splits_dict['Simulation_10_10000']['T_train'], 
                                       splits_dict['Simulation_10_10000']['X_train'], splits_dict['Simulation_10_10000']['X_test'],
                                       splits_dict['Simulation_10_10000']['true_cate_test'])



In [ ]:
plot_predict_cate_corr(estimated_cate_dml, splits_dict['Simulation_10_10000']['true_cate_test'])

Non-Parametric DML

In [ ]:
def non_param_dml(Y_train: pd.DataFrame, T_train: pd.DataFrame, X_train: pd.DataFrame, X_test: pd.DataFrame, true_cate_test: pd.DataFrame):

    Y_train = Y_train.to_numpy()
    T_train = T_train.to_numpy()
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    true_cate_test = true_cate_test.to_numpy()

    est_npm = NonParamDML(
        model_y='auto',
        model_t=DummyClassifier(),
        model_final=GradientBoostingRegressor(),
        cv = 5,
        random_state = 42,
        discrete_treatment=True,
    )
    
    est_npm.fit(Y=Y_train, T=T_train, X=X_train, W=None, cache_values=True)
    estimated_cate = est_npm.effect(X_test)
    mse = mean_squared_error(true_cate_test, estimated_cate)

    return estimated_cate, mse

estimated_cate_non_param, non_param_MSE = non_param_dml(splits_dict['Simulation_10_10000']['Y_train'], splits_dict['Simulation_10_10000']['T_train'], 
                                       splits_dict['Simulation_10_10000']['X_train'], splits_dict['Simulation_10_10000']['X_test'],
                                       splits_dict['Simulation_10_10000']['true_cate_test'])



In [ ]:
plot_predict_cate_corr(estimated_cate_non_param, splits_dict['Simulation_10_10000']['true_cate_test'])

In [ ]:
print(f'MSE T-Learner: {T_MSE}')
print(f'MSE Double ML: {cf_dml_MSE}')
print(f'MSE Non-Parametric Double ML: {non_param_MSE}')